In [116]:
import requests

The API key “demo” below is a demo key which only works for obtaining data for “KOSS”. For other tickers, you need to sign up for an account at https://financialmodelingprep.com/developer to get an API key for free (for the first 250 requests, however you can actually just sign up for another account when your free requests are used up).

In [146]:
apikey = "4f9235dd83eb59bf340f1ed5d3612cc9"
company = "KOSS"

In [153]:
def valuecompany(quote):
    #Latest dividend of the company
    dividend = requests.get(f'https://financialmodelingprep.com/api/v3/financials/income-statement/{quote}?apikey={apikey}')
    dividend = dividend.json()
    dividend = dividend['financials']
    try:
        Dtoday = float(dividend[0]['Dividend per Share'])
    except:
        Dtoday = 0.0
    
    #ROE and PAyout Ratio
    metrics = requests.get(f'https://financialmodelingprep.com/api/v3/company-key-metrics/{quote}?apikey={apikey}')
    metrics = metrics.json()

    ROE = float(metrics['metrics'][0]['ROE'])
    payout_ratio = float(metrics['metrics'][0]['Payout Ratio'])

    sustgrwothrate = ROE*(1-payout_ratio)
    print(sustgrwothrate)
    
     ##Cost of equity:

    #Risk Free Rate

    import pandas as pd
    #if you get an error after executing the code, try adding below:
    pd.core.common.is_list_like = pd.api.types.is_list_like

    import pandas_datareader.data as web
    import datetime

    start = datetime.datetime(2020, 1, 1)
    end = datetime.datetime(2021, 5, 30)

    Treasury = web.DataReader(['TB1YR'], 'fred', start, end)
    RF = float(Treasury.iloc[-1])
    RF = RF/100
    
    #Beta
    beta = requests.get(f'https://financialmodelingprep.com/api/v3/company/profile/{quote}?apikey={apikey}')
    beta = beta.json()
    beta = float(beta['profile']['beta'])
    
    #Market Return
    start = datetime.datetime(2020, 1, 1)
    end = datetime.datetime(2021, 5, 30)

    SP500 = web.DataReader(['sp500'], 'fred', start, end)
    #Drop all Not a number values using drop method.
    SP500.dropna(inplace = True)

    SP500yearlyreturn = (SP500['sp500'].iloc[-1]/ SP500['sp500'].iloc[0])-1
   
    ke = RF+(beta*(SP500yearlyreturn - RF))
    
    DDM = (Dtoday*(1+sustgrwothrate))/(ke-sustgrwothrate)
    print(DDM)
    return DDM

In [155]:
valuecompany('JNJ')

0.06689528746167704
0.0


0.0